In [1]:
import PySimpleGUI as sg
import lightkurve as lk
import numpy as np
import pandas as pd
import os

# Versão com janelas múltiplas

In [5]:
######################
#  FUNCIONALIDADES   #
######################
# Lista todos os setores observados para o alvo especificado em 'Target'
def search(tic):
    sector_list = []
    search = lk.search_lightcurve(target=f'TIC {tic}', author='SPOC', exptime=120)
    for i in range(len(search)):
        sector_list.append(search.table['mission'][i][12:])
    return sector_list

# Faz o download dos setores selecionados na segunda janela para o alvo especificado em "Target"
def download_sectors(selected_sectors):
    try:
        os.mkdir('dat')
    except FileExistsError:
        pass
    for i in range(len(selected_sectors)):
        search_1 = lk.search_lightcurve(target=f'TIC {tic}', sector=selected_sectors[i], author='SPOC', exptime=120)
        lc = search_1.download().normalize().remove_outliers(sigma=3.5)
        time = lc.time.value
        flux = lc.flux.value
        data = pd.DataFrame(np.column_stack([time,flux]), columns=['Time', 'Flux'])
        save_file = data.to_csv(f'dat/{tic.zfill(10)}_{selected_sectors[i].zfill(3)}.dat', sep=' ', index=None, header=None)
    return save_file

# Faz o download de todos os setores observados dos alvos especificados no arquivo (deve ser um arquivo .csv ou .txt)
def download_list_all(file_address):
    try:
        os.mkdir('dat')
    except FileExistsError:
        pass
    file = pd.read_csv(file_address, sep=';')
    id_list = file.TIC
    for i in range(len(id_list)):
        tic = id_list[i]
        search = lk.search_lightcurve(target=f'TIC {tic}', author='SPOC', exptime=120)
        for j in range(len(search)):
            sector = search.table['mission'][j][12:]
            lc = search[j].download().normalize().remove_outliers(sigma=3.5)
            time = lc.time.value
            flux = lc.flux.value
            data = pd.DataFrame(np.column_stack([time,flux]), columns=['Time', 'Flux'] )
            save_file = data.to_csv(f'dat/{str(tic).zfill(10)}_{str(sector).zfill(3)}.dat', sep=' ', index=None, header=None)
    return save_file

# Faz o download dos setores dentro do range especificado em 'Start' e 'Final' para todos os alvos especificados no arquivo (deve ser um arquivo .csv ou .txt)
def download_list_sec(file_address, start_sec, final_sec):
    try:
        os.mkdir('dat')
    except FileExistsError:
        pass
    file = pd.read_csv(file_address, sep=';')
    id_list = file.TIC
    for i in range(len(id_list)):
        tic = id_list[i]
        search = lk.search_lightcurve(target=f'TIC {tic}', author='SPOC', exptime=120)
        for j in range(len(search)):
            sector = search.table['mission'][j][12:]
            if int(start_sec) < int(sector) < int(final_sec):
                lc = search[j].download().normalize().remove_outliers(sigma=3.5)
                time = lc.time.value
                flux = lc.flux.value
                data = pd.DataFrame(np.column_stack([time,flux]), columns=['Time', 'Flux'])
                save_file = data.to_csv(f'dat/{str(tic).zfill(10)}_{str(sector).zfill(3)}.dat', sep=' ', index=None, header=None)
    return save_file


# Faz o download de todos os setores concatenados em uma única LC (merged) para o alvo especificado em 'Target'
def download_merged(tic):
    try:
        os.mkdir('merged')
    except FileExistsError:
        pass
    search = lk.search_lightcurve(target=f'TIC {tic}', author='SPOC', exptime=120)
    lc = search.download_all().stitch().remove_outliers(sigma=3.5)
    time = lc.time.value
    flux = lc.flux.value
    data = pd.DataFrame(np.column_stack([time,flux]), columns=['Time', 'Flux'] )
    save_file = data.to_csv(f'merged/{str(tic).zfill(10)}_merged.dat', sep=' ', index=None, header=None)
    
    return save_file


# Faz o download de todos os setores concatenados em uma única LC (merged) para todos os alvos especificados no arquivo (deve ser um arquivo .csv ou .txt)
def download_merged_all(file_address):
    try:
        os.mkdir('merged')
    except FileExistsError:
        pass
    file = pd.read_csv(file_address, sep=';')
    id_list = file.TIC
    for i in range(len(id_list)):
        tic = id_list[i]
        search = lk.search_lightcurve(target=f'TIC {tic}', author='SPOC', exptime=120)
        lc = search.download_all().stitch().remove_outliers(sigma=3.5)
        time = lc.time.value
        flux = lc.flux.value
        data = pd.DataFrame(np.column_stack([time,flux]), columns=['Time', 'Flux'] )
        save_file = data.to_csv(f'merged/{str(tic).zfill(10)}_merged.dat', sep=' ', index=None, header=None)
        
    return save_file

##########################
#   LAYOUT DAS JANELAS   #
##########################
def main_window():
    sg.theme('DarkGray1')
    init_directory = 'C:/'
    layout = [
        [sg.Text('Target:', font=('Cambria', 14)), sg.Input('', key='-TARGET_INPUT-', size=(51, 0)), sg.Button('Search', key='-SEARCH_BUTTON-', size=(12, 0))],
        [sg.HSeparator()],
        [sg.Text('List of targets:', font=('Cambria', 14)), sg.Input(key='-TARGET_FILE-'), 
         sg.FileBrowse(initial_folder=init_directory, file_types=[('Text Files', '*.txt'), ('CSV Files', '*.csv')], enable_events=True, size=(12, 0))],
        [sg.Radio('All sectors', 'sec', font=('Cambria', 14), key='-ALL_SECTORS_RAD-', default=True, enable_events=True)],
        [sg.Radio('Choose sectors range:', 'sec', font=('Cambria', 14), key='-SECTORS_RAD-', enable_events=True), sg.Text('Start:', font=('Cambria', 14)), 
         sg.Input('', key='-START_INPUT-', disabled=True, disabled_readonly_background_color='gray', size=(5, 0)), 
         sg.Text('Final:', font=('Cambria', 14)), sg.Input('', key='-FINAL_INPUT-', disabled=True, disabled_readonly_background_color='gray', size=(5, 0))],
        [sg.HSeparator()],
        [sg.Push(), sg.Checkbox('All Sectors (MERGED)', key='-MERGED-', font=('Cambria', 14)), sg.Push()],
        [sg.HSeparator()],
        [sg.Text('')],
        [sg.Push(), sg.Button('DOWNLOAD', key='-DOWNLOAD_BUTTON_1-', size=(12, 2)), sg.Push()]
    ]
    return sg.Window('TLC Download Project', layout=layout, size=(550, 300), finalize=True)

def sectors_window():
    sg.theme('DarkGray1')
    layout = [
        [sg.Push(), sg.Button('Download merged', key='-MERGED_BUTTON-'), sg.Push()],
        [sg.Text('Observed Sectors:', font=('Cambria', 14))],
        [sg.Push(), sg.Listbox(data, size=(5, 10), select_mode=sg.LISTBOX_SELECT_MODE_MULTIPLE, key='-LIST-', enable_events=True), sg.Push()],
        [sg.Button('Download', key='-DOWNLOAD_BUTTON_2-'), sg.Push(), sg.Button('Cancel', key='-CANCEL_BUTTON-')]
              
    ]
    return sg.Window('List of Sectors', layout=layout, size=(250, 300), finalize=True)

############################
#   CRIAÇÃO DAS JANELAS   #
###########################
window_1, window_2 = main_window(), None

while True:
    window, event, values = sg.read_all_windows()
    
    if window == window_1 and event == sg.WIN_CLOSED:
        break
        
    elif window == window_1 and event == '-SEARCH_BUTTON-':
        tic = values['-TARGET_INPUT-']
        data = search(tic)
        window_2 = sectors_window()
        
    elif window == window_2 and event == '-DOWNLOAD_BUTTON_2-':
        selected_sectors = values['-LIST-']
        download_sectors(selected_sectors)
        sg.popup('Files saved successfully!')
    
    elif window == window_2 and event == '-MERGED_BUTTON-':
        download_merged(tic)
        sg.popup('Files saved successfully!')
        
    elif window == window_2 and event == sg.WIN_CLOSED:
        window_2.hide()
        
    elif window == window_2 and event == '-CANCEL_BUTTON-':
        window_2.hide()
        
    elif window == window_1 and values['-ALL_SECTORS_RAD-'] == True:
        window_1['-START_INPUT-'].update(disabled=True)
        window_1['-FINAL_INPUT-'].update(disabled=True)
        if event == '-DOWNLOAD_BUTTON_1-':
            if values['-MERGED-'] == False:
                file_address = values['-TARGET_FILE-']
                download_list_all(file_address)
                sg.popup('Files saved successfully!')
            
            elif values['-MERGED-'] == True:
                file_address = values['-TARGET_FILE-']
                download_merged_all(file_address)
                sg.popup('Files saved successfully!')
            
    elif window == window_1 and values['-SECTORS_RAD-'] == True:
        window_1['-START_INPUT-'].update(disabled=False)
        window_1['-FINAL_INPUT-'].update(disabled=False)
        if event == '-DOWNLOAD_BUTTON_1-':
            file_address = values['-TARGET_FILE-']
            start_sec = values['-START_INPUT-']
            final_sec = values['-FINAL_INPUT-']
            download_list_sec(file_address, start_sec, final_sec)
            sg.popup('Files saved successfully!')
            

window.close()

In [ ]:
layout = [
    [sg.Radio('Option 1', 'rad' , key='-OPTION_1-' , default=True, enable_events=True)],
    [sg.Radio('Option 2', 'rad' , key='-OPTION_2-', enable_events=True), 
     sg.Text('Box 1', key='-BOX_1-'), sg.Input('', key='-INPUT_1-', disabled=True, disabled_readonly_background_color='gray', size=(5, 0)),
     sg.Text('Box 2', key='-BOX_2-'), sg.Input('', key='-INPUT_2-', disabled=True, disabled_readonly_background_color='gray', size=(5, 0))]
]

window = sg.Window('TESTE', layout=layout)
while True:
    event, values = window.read()
    print(event, values)
    if event == None or event == sg.WIN_CLOSED:
        break
    elif values['-OPTION_2-'] == True:
        window['-INPUT_1-'].update(disabled=False)
        window['-INPUT_2-'].update(disabled=False)
        #sg.popup('EVENTO FUNCIONA!!!')
    elif values['-OPTION_1-'] == True:
        window['-INPUT_1-'].update(disabled=True)
        window['-INPUT_2-'].update(disabled=True)

window.read()
window.close()

In [ ]:
####### THEME #######
sg.theme('DarkGray1')
#####################

#LAYOUT
layout = [
    [sg.Radio('Target', 'tgt' , key='-TARGET_RADIO-', default=True), sg.Input('', key='-TARGET_INPUT-', size=(45, 0), pad=(51, 5))],
    [sg.Radio('List of Targets', 'tgt', key='-TARGET_LIST_RADIO-'), sg.Input(key='-TARGET_FILE-'), sg.FileBrowse(enable_events=True, size=(12, 0))],
    [sg.HSeparator()],
    [sg.Radio('Sector', 'sct', key='-SEC_RADIO-', default=True), sg.Input('', key='-SEC_INPUT-', size=(10, 0), pad=(51, 5))],
    [sg.Radio('List of Sectors', 'sct', key='-SEC_LIST_RADIO-'), sg.Input('', key='-SEC_LIST_INPUT-', pad=(5, 5))],
    [sg.Radio('All Sectors (merged)', 'sct', key='-ALL_SEC_RADIO-')],
    [sg.HSeparator()],
    [sg.Push(), sg.Button('DOWNLOAD', key='-DOWNLOAD_BUTTON-', size=(12, 2)), sg.Push()],
    [sg.ProgressBar(100, size=(500, 15), bar_color=('green', 'lightgrey'))]
    
]

# FUNCTIONALITIES
def single_target(tic, sector):
    search = lk.search_lightcurve(target=f'TIC {tic}', sector=sector, author='SPOC', exptime=120)
    lc = search.download().normalize().remove_outliers(sigma=3.5)
    time = lc.time.value
    flux = lc.flux.value
    df = pd.DataFrame(np.column_stack([time,flux]), columns=['Time', 'Flux'] )
    return df
    


# JANELA
window = sg.Window('TL Download Project', layout=layout, size=(550, 300))


while True:
    event, value = window.read()
    
    if event == sg.WIN_CLOSED:
        break
        
    if event == '-DOWNLOAD_BUTTON-':
        ### Download the specified sector of the specified target ###
        if value['-TARGET_RADIO-'] == True and value['-SEC_RADIO-'] == True:
            tic = value['-TARGET_INPUT-']
            sector = value['-SEC_INPUT-']
            data = single_target(tic, sector)
            data.to_csv(f'{tic.zfill(10)}_{str(sector).zfill(2)}.dat', sep=' ', index=None, header=None)
            
            sg.popup('File saved successfully!')
            
        
window.close()

In [ ]:
def search(tic):
    sector_list = []
    search = lk.search_lightcurve(target=f'TIC {tic}', author='SPOC', exptime=120)
    for i in range(len(search)):
        sector_list.append(search.table['mission'][i][12:])
    
    return sector_list

In [ ]:
data = search(137608661)
data